### Tips for  Webscraping Updated Table in Week3 Peer Graded Assignment

 **After retreiving the URL and creating a Beautiful soup object** 

 **Firstly create a list**  

 **Later after finding the table and table data  create a dictionary called cell having 3 keys PostalCode, Borough and Neighborhood.**

**As postal code contains upto 3 characters extract that using tablerow.p.text**

 **Next use split ,strip and replace functions for getting Borough and Neighborhood information.**.

 **Append to the list**  

 **Create a dataframe with list**

## Sample code

```python
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
```

#### link: https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969

In [1]:
import pandas as pd
import requests

In [2]:
# import list of postal codes of Canada M (Toronto)
url = 'https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969'
page = requests.get(url)

In [3]:
# table to data frame
df = pd.read_html(page.content,keep_default_na=False)[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
df.shape

(180, 3)

In [5]:
# to ignoring cells with "Not assigned" Borough
df.drop(df[ df['Borough'] == 'Not assigned' ].index, inplace=True)

# Grouping Postal Code and Borough; separate Neighbourhood with commas 
df.groupby(['Postal Code','Borough'])['Neighbourhood'].apply(lambda x: ','.join(x.astype(str))).reset_index()
df

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
160,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
165,M4Y,Downtown Toronto,Church and Wellesley
168,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
169,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
# no cases of Neighbourdhood = Not assigned
print(df[df['Neighbourhood'] == 'Not assigned'])

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []


In [7]:
df.shape

(103, 3)

## Part 2

In [8]:
df_latlng = pd.DataFrame(df['Postal Code'], columns=['Postal Code'])

df_latlng.head()

,Postal Code
2,M3A
3,M4A
4,M5A
5,M6A
6,M7A


In [9]:
df_coord = pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Merge data frame with coordinates and data frame principal
df = pd.merge(df,
             df_coord,
             on = 'Postal Code',
             how = 'left')

df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


## Part 3

In [11]:
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation


!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize


! pip install folium==0.5.0
import folium # plotting library

In [12]:
# Data frame only with Toronto's Borough
df_Toronto = df[df['Borough'].str.contains('Toronto')]

# reset index
df_Toronto.reset_index(drop=True)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031
5,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
6,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
7,M6G,Downtown Toronto,Christie,43.669542,-79.422564
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.650571,-79.384568
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259


## >>>>>>>>
#### Apagar credenciais antes de disponibilizar
## <<<<<<<<

In [13]:
# Creditials Foursquare
CLIENT_ID = '3CSGBPR2BHRO1NR4RWXVUYEVB2HSY410GFTWYBZZC3L24NVE' # my Foursquare ID
CLIENT_SECRET = 'STXK25ZGHI4KO2FEB4EKWHCI5EYKHERXYEDDL1BK0U031ZWR' # my Foursquare Secret
ACCESS_TOKEN = 'N4SUPTNFNC00F34HLOR3FOKYNXXJ4Z15E2QZWTWFJ2OE1QD1' # my FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

#### Let's assume we are searching for pharmacies nearby the Hotel _Victoria's Mansion Guest House_, 68 Gloucester Street, Toronto, ON, Canada

In [18]:
# Get the coorditates Hotel.
address = '68 Gloucester Street, Toronto, ON, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6674676 -79.3822102


In [19]:
search_query = 'Pharmacy'
radius = 500
print(search_query + ' .... OK!')

Pharmacy .... OK!


In [20]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

results = requests.get(url).json()

In [21]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-21-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4c45dc70dd1f2d7fd0d381f9,Pace Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1626544940,False,14 Isabella St.,Yonge St,43.667751,-79.385186,"[{'label': 'display', 'lat': 43.667751, 'lng':...",241,M4Y 1N1,CA,Toronto,ON,Canada,"[14 Isabella St. (Yonge St), Toronto ON M4Y 1N..."
1,517c0441e4b0eef5740b871a,Maple Leaf Medical Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1626544940,False,14 College St,Yonge St,43.665615,-79.384352,"[{'label': 'display', 'lat': 43.665615, 'lng':...",268,NaN,CA,Toronto,ON,Canada,"[14 College St (Yonge St), Toronto ON, Canada]"
2,4c7d727cd461a143b62f7df5,The Village Pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1626544940,False,473 Church St,Maitland,43.664439,-79.380263,"[{'label': 'display', 'lat': 43.66443931976355...",371,M3B 2H1,CA,Toronto,ON,Canada,"[473 Church St (Maitland), Toronto ON M3B 2H1,..."
3,5ae76f0f31fd14002c08909a,Loblaw pharmacy,"[{'id': '4bf58dd8d48988d10f951735', 'name': 'P...",v-1626544940,False,60 Carlton St,NaN,43.662284,-79.380791,"[{'label': 'display', 'lat': 43.66228360915324...",588,M5B 1J2,CA,Toronto,ON,Canada,"[60 Carlton St, Toronto ON M5B 1J2, Canada]"


In [22]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Pace Pharmacy,Pharmacy,14 Isabella St.,Yonge St,43.667751,-79.385186,"[{'label': 'display', 'lat': 43.667751, 'lng':...",241,M4Y 1N1,CA,Toronto,ON,Canada,"[14 Isabella St. (Yonge St), Toronto ON M4Y 1N...",4c45dc70dd1f2d7fd0d381f9
1,Maple Leaf Medical Pharmacy,Pharmacy,14 College St,Yonge St,43.665615,-79.384352,"[{'label': 'display', 'lat': 43.665615, 'lng':...",268,NaN,CA,Toronto,ON,Canada,"[14 College St (Yonge St), Toronto ON, Canada]",517c0441e4b0eef5740b871a
2,The Village Pharmacy,Pharmacy,473 Church St,Maitland,43.664439,-79.380263,"[{'label': 'display', 'lat': 43.66443931976355...",371,M3B 2H1,CA,Toronto,ON,Canada,"[473 Church St (Maitland), Toronto ON M3B 2H1,...",4c7d727cd461a143b62f7df5
3,Loblaw pharmacy,Pharmacy,60 Carlton St,NaN,43.662284,-79.380791,"[{'label': 'display', 'lat': 43.66228360915324...",588,M5B 1J2,CA,Toronto,ON,Canada,"[60 Carlton St, Toronto ON M5B 1J2, Canada]",5ae76f0f31fd14002c08909a


In [23]:
dataframe_filtered.name

0                  Pace Pharmacy
1    Maple Leaf Medical Pharmacy
2           The Village Pharmacy
3                Loblaw pharmacy
Name: name, dtype: object

In [24]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Victoria's Mansion Guest House

# add a red circle marker to represent the Victoria's Mansion Guest House
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Victoria s Mansion Guest House',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Pharmacies as blue circle markers
for lat, lng, name in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.name):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=name,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##### Let's explore the closest pharmacy: Pace Pharmacy

In [32]:
venue_id = '4c45dc70dd1f2d7fd0d381f9' # ID of Pace Pharmacy
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
url

'https://api.foursquare.com/v2/venues/4c45dc70dd1f2d7fd0d381f9?client_id=3CSGBPR2BHRO1NR4RWXVUYEVB2HSY410GFTWYBZZC3L24NVE&client_secret=STXK25ZGHI4KO2FEB4EKWHCI5EYKHERXYEDDL1BK0U031ZWR&oauth_token=N4SUPTNFNC00F34HLOR3FOKYNXXJ4Z15E2QZWTWFJ2OE1QD1&v=20180604'

In [33]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'likes', 'like', 'dislike', 'ok', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'reasons', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'seasonalHours', 'pageUpdates', 'inbox', 'attributes'])


{'id': '4c45dc70dd1f2d7fd0d381f9',
 'name': 'Pace Pharmacy',
 'contact': {'phone': '+14165157223',
  'formattedPhone': '+1 416-515-7223',
  'twitter': 'pacepharmacy'},
 'location': {'address': '14 Isabella St.',
  'crossStreet': 'Yonge St',
  'lat': 43.667751,
  'lng': -79.385186,
  'labeledLatLngs': [{'label': 'display',
    'lat': 43.667751,
    'lng': -79.385186}],
  'postalCode': 'M4Y 1N1',
  'cc': 'CA',
  'city': 'Toronto',
  'state': 'ON',
  'country': 'Canada',
  'formattedAddress': ['14 Isabella St. (Yonge St)',
   'Toronto ON M4Y 1N1',
   'Canada']},
 'canonicalUrl': 'https://foursquare.com/v/pace-pharmacy/4c45dc70dd1f2d7fd0d381f9',
 'categories': [{'id': '4bf58dd8d48988d10f951735',
   'name': 'Pharmacy',
   'pluralName': 'Pharmacies',
   'shortName': 'Pharmacy',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pharmacy_',
    'suffix': '.png'},
   'primary': True}],
 'verified': False,
 'stats': {'tipCount': 6},
 'likes': {'count': 0, 'groups': []},
 'like'

##### Get number of tips from Pace Pharmacy

In [34]:
result['response']['venue']['tips']['count']

6

In [35]:
## Ecco Tips
limit = 15 # set limit to be greater than or equal to the total number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION, limit)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60f31d75f1fb0416ec7c39aa'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'tips': {'count': 6,
   'items': [{'id': '4cdeaff6abfb2c0fa1e59a5a',
     'createdAt': 1289662454,
     'text': "great little pharmacy.  very clean store with decent selection for it's size.  and they have a cool lab where they mix up drugs with a window so you can watch.  a much needed change from the competiti",
     'type': 'user',
     'canonicalUrl': 'https://foursquare.com/item/4cdeaff6abfb2c0fa1e59a5a',
     'likes': {'count': 0, 'groups': []},
     'like': False,
     'logView': True,
     'agreeCount': 0,
     'disagreeCount': 0,
     'todo': {'count': 0},
     'user': {'id': '4603897',
      'firstName': 'Stephen',
      'lastName': 'P',
      'gender': 'male',
      'countryCode': 'CA',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/blank_boy.png',
       'default': True}}}]}}}

In [36]:
tips = results['response']['tips']['items']

tip = results['response']['tips']['items'][0]
tip.keys()

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])

##### Show the tips

In [37]:
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered.reindex()

<ipython-input-37-252e55242536>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)
<ipython-input-37-252e55242536>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  tips_df = json_normalize(tips) # json normalize tips


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.id
0,great little pharmacy. very clean store with decent selection for it's size. and they have a cool lab where they mix up drugs with a window so you can watch. a much needed change from the competiti,0,0,4cdeaff6abfb2c0fa1e59a5a,Stephen,P,4603897
